<a href="https://colab.research.google.com/github/indl1670/PSMaker/blob/main/Polygon_Blurring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Detectron2 설치

In [ ]:
!python -m pip install pyyaml==5.1
# Detectron2 has not released pre-built binaries for the latest pytorch (https://github.com/facebookresearch/detectron2/issues/4053)
# so we install from source instead. This takes a few minutes.
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

# Install pre-built detectron2 that matches pytorch version, if released:
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
#!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/{CUDA_VERSION}/{TORCH_VERSION}/index.html

# exit(0)  # After installation, you may need to "restart runtime" in Colab. This line can also restart runtime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 274 kB 29.8 MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44092 sha256=1ebe243fe55a3c0770fc5d66a6ddc806c84be8b56ba1175202537a0b1a155a7a
  Stored in directory: /root/.cache/pip/wheels/77/f5/10/d00a2bd30928b972790053b5de0c703ca87324f3fead0f2fd9
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-qkfw8ume
  Running command git clone -q https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-qkfw8ume
     |████████████████████████████████| 50 kB 7.1 MB/s 
     |████████████████████████████████| 79 kB

# 버전 확인

In [ ]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
torch:  1.11 ; cuda:  cu113
detectron2: 0.6


# Train

## 모듈 로드

In [ ]:
# If error occured, check current directory
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer
from detectron2.engine import DefaultPredictor
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.utils.visualizer import Visualizer
from detectron2.utils.visualizer import ColorMode
from detectron2.config import get_cfg
from detectron2 import model_zoo
from google.colab.patches import cv2_imshow

import numpy as np
import random
import cv2
import matplotlib.pyplot as plt
import os
import pickle
import json
import random

## 파라미터 설정

In [ ]:
# 사전학습된 모델 파일 경로 - 수정x
# pre-trained model file
config_file_path = "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"
checkpoint_url = "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"

# 학습이 완료된 모델을 저장시킬 경로 입력
# test result directory
output_dir = "./drive/MyDrive/졸업과제/output2/"

# 카테고리 개수 입입력
# Enter the total number of objects
num_classes = 4

# 학습에 사용될 장치 설정 - 수정 x
# Use cuda device
device = "cuda"

# 데이터셋 이름, 이미지 경로, 라벨링 경로 입력
# Dataset configuration
train_dataset_name = "train_" # 학습 데이터셋 이름 입력(register_coco에 한번 저장된 이름일 경우 에러를 유발하기 때문에 등록할때마다 다른 이름 작성 필요)
train_images_path = "./drive/MyDrive/졸업과제/Images2/" # 학습 데이터셋 이미지 경로 입력
train_json_annot_path = "./drive/MyDrive/졸업과제/annotations2.json" # 학습 데이터셋 라벨링 경로 + 라벨링 파일 이름 입력

test_dataset_name ="valid_"
test_images_path = "./drive/MyDrive/졸업과제/Images2"
test_json_annot_path = "./drive/MyDrive/졸업과제/annotations2.json"

# 학습 진행시 configure파일을 저장시킬 폴더 경로 입력 + configure파일 이름(학습마다 다르게 설정) 입력
# Save configuration file
cfg_save_path = "./drive/MyDrive/졸업과제/cfg_file/0630.pickle"

## 데이터셋 등록

In [ ]:
#데이터셋의 이름 변경 필요
# Reginster train, test dataset
register_coco_instances(name = train_dataset_name, metadata = {}, json_file = train_json_annot_path, image_root = train_images_path)
register_coco_instances(name = test_dataset_name, metadata = {}, json_file = test_json_annot_path, image_root = test_images_path)

In [ ]:
# 샘플 이미지 출력
# Draw samples
def plot_samples(dataset_name, n=1):
    dataset_custom = DatasetCatalog.get(dataset_name)
    dataset_custom_metadata = MetadataCatalog.get(dataset_name)
    
    for s in random.sample(dataset_custom, n):
        img = cv2.imread(s["file_name"])
        v = Visualizer(img[:,:,::-1], metadata=dataset_custom_metadata, scale=0.5)
        v = v.draw_dataset_dict(s)
        plt.figure(figsize=(15, 20))
        plt.imshow(v.get_image())
        plt.show()

# 디텍트론 학습
# Set configuration    
def get_train_cfg(config_file_path, checkpoint_url, train_dataset_name, test_dataset_name, num_classes, device, output_dir):
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file(config_file_path))
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(checkpoint_url)
    cfg.DATASETS.TRAIN = (train_dataset_name,)
    cfg.DATASETS.TEST = (test_dataset_name,)
    
    cfg.DATALOADER.NUM_WORKERS = 2
    cfg.SOLVER.IMS_PER_BATCH = 2
    cfg.SOLVER.BASE_LR = 0.001
    # epoch = MAX_ITER * BATCH_SIZE / TOTAL_NUM_IMAGES
    cfg.SOLVER.MAX_ITER = 5000 # 적절한 iteration값 수정 필요
    cfg.SOLVER.STEPS = []
    
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = num_classes
    cfg.MODEL.DEVICE = device
    cfg.OUTPUT_DIR = output_dir
    
    return cfg

# 이미지 detect 함수
# 개별 단일 이미지 detect 함수
def on_image(image_path, predictor):
    im = cv2.imread(image_path)
    outputs = predictor(im)
    v = Visualizer(im[:,:,::-1], metadata={}, scale=0.5)
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    plt.figure(figsize=(14,10))
    plt.imshow(v.get_image())
    plt.show()

# 폴더 내 전체 이미지 detect및 저장 함수
def on_image_All(image_path, predictor):
    im = cv2.imread(image_path)
    outputs = predictor(im)
    v = Visualizer(im[:,:,::-1], metadata={}, scale=0.5)
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))

    return (v.get_image()) # Save multi images

# 객체 블러링
def direct_cv_blur_detector(image_path, predictor):
    im = cv2.imread(image_path)
    blur_img = cv2.GaussianBlur(im, (0, 0), 50)
    outputs = predictor(im)
    x = outputs['instances'].to('cpu')
    instances = outputs["instances"]

    #원하는 값 얻는 방법.
    mask = x.get('pred_masks').permute(1, 2, 0).to("cpu").numpy()
    num_instances = mask.shape[2]
    mask_array_instance = []

    for i in range(num_instances):
      mask_array_instance.append(mask[:, :, i:(i+1)])
      if str(instances.pred_classes[i])[7:8] == '3':
        img = np.where(mask_array_instance[i] == True, 0, 1)
        array_img = np.asarray(img)

        #아래부분을 리스트로 받아서 풀면 더 빠르지 않을까 생각은 해봄. -> 0으로 함으로써 사용 x
        im[np.where((array_img==0).all(axis=2))]= 0
      else:
        pass
    return np.where(im == 0, blur_img, im)

## 학습

In [ ]:
def main():
    cfg = get_train_cfg(config_file_path, checkpoint_url, train_dataset_name,
                        test_dataset_name, num_classes, device, output_dir)

    with open(cfg_save_path, 'wb') as f:
        pickle.dump(cfg, f, protocol=pickle.HIGHEST_PROTOCOL)

    os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

    trainer = DefaultTrainer(cfg)
    trainer.resume_or_load(resume=False)

    trainer.train()
    
if __name__ == '__main__':
    main()

[06/30 01:41:31 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

model_final_f10217.pkl: 178MB [01:38, 1.80MB/s]                           
Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (5, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (5,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (16, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (16,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to

[06/30 01:43:17 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.7/dist-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[06/30 01:43:30 d2.utils.events]:  eta: 0:49:37  iter: 19  total_loss: 2.486  loss_cls: 1.519  loss_box_reg: 0.1002  loss_mask: 0.6924  loss_rpn_cls: 0.1891  loss_rpn_loc: 0.03629  time: 0.6142  data_time: 0.1043  lr: 1.9981e-05  max_mem: 2673M
[06/30 01:43:43 d2.utils.events]:  eta: 0:51:30  iter: 39  total_loss: 2.215  loss_cls: 1.208  loss_box_reg: 0.152  loss_mask: 0.6897  loss_rpn_cls: 0.1023  loss_rpn_loc: 0.03072  time: 0.6354  data_time: 0.1043  lr: 3.9961e-05  max_mem: 2673M
[06/30 01:43:57 d2.utils.events]:  eta: 0:51:59  iter: 59  total_loss: 1.605  loss_cls: 0.6139  loss_box_reg: 0.1432  loss_mask: 0.6855  loss_rpn_cls: 0.06907  loss_rpn_loc: 0.02166  time: 0.6464  data_time: 0.1183  lr: 5.9941e-05  max_mem: 2673M
[06/30 01:44:10 d2.utils.events]:  eta: 0:51:46  iter: 79  total_loss: 1.451  loss_cls: 0.4138  loss_box_reg: 0.2625  loss_mask: 0.6794  loss_rpn_cls: 0.06289  loss_rpn_loc: 0.0235  time: 0.6480  data_time: 0.0871  lr: 7.9921e-05  max_mem: 2673M
[06/30 01:44:23 d2

In [ ]:
with open("./drive/MyDrive/졸업과제/cfg_file/0630.pickle", 'rb') as f:
    cfg = pickle.load(f)
# cfg = get_train_cfg(config_file_path, checkpoint_url, train_dataset_name, test_dataset_name, num_classes, device, output_dir)   
cfg.MODEL.WEIGHTS = "./drive/MyDrive/졸업과제/output2/model_final.pth"
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5

predictor = DefaultPredictor(cfg)

[06/30 02:56:24 d2.checkpoint.c2_model_loading]: Following weights matched with model:
| Names in Model                                  | Names in Checkpoint                                                                                  | Shapes                                          |
|:------------------------------------------------|:-----------------------------------------------------------------------------------------------------|:------------------------------------------------|
| backbone.bottom_up.res2.0.conv1.*               | backbone.bottom_up.res2.0.conv1.{norm.bias,norm.running_mean,norm.running_var,norm.weight,weight}    | (64,) (64,) (64,) (64,) (64,64,1,1)             |
| backbone.bottom_up.res2.0.conv2.*               | backbone.bottom_up.res2.0.conv2.{norm.bias,norm.running_mean,norm.running_var,norm.weight,weight}    | (64,) (64,) (64,) (64,) (64,64,3,3)             |
| backbone.bottom_up.res2.0.conv3.*               | backbone.bottom_up.res2.0.conv3.{norm.bia

In [ ]:
save_path = "./drive/MyDrive/졸업과제/result/"
img_path = "./drive/MyDrive/Graduation/video_images"
image_list = os.listdir(img_path)
image_list = [file for file in image_list if file.endswith(".jpg")]

for i in range(len(image_list)):
    image_path = os.path.join(img_path, image_list[i])
    #result = on_image(image_path, predictor) # Normal
    result = direct_cv_blur_detector(image_path, predictor) # Blur
    cv2.imwrite(save_path + image_list[i], result)

In [ ]:
save_path = "./drive/MyDrive/졸업과제/TestResult/"
img_path = "./drive/MyDrive/졸업과제/TestImages"
image_list = os.listdir(img_path)
image_list = [file for file in image_list if file.endswith(".png")]

for i in range(len(image_list)):
    image_path = os.path.join(img_path, image_list[i])
    #result = on_image(image_path, predictor) # Normal
    result = direct_cv_blur_detector(image_path, predictor) # Blur
    cv2.imwrite(save_path + image_list[i], result)